## Main

In [ ]:
import gc
import copy
import math
import random
import numpy as np
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from typing import Dict, Any, Tuple
import optuna
from optuna.pruners import HyperbandPruner#, SuccessiveHalvingPruner
from sklearn.ensemble import VotingClassifier
from sklearn import set_config
from modules.labeling_lib import get_prices
from modules.labeling_lib import get_features
from modules.labeling_lib import get_labels_one_direction
from modules.labeling_lib import sliding_window_clustering
from modules.tester_lib import test_model_one_direction
from modules.export_lib import export_model_to_ONNX
from modules.export_lib import XGBWithEval
from modules.export_lib import LGBMWithEval
from modules.export_lib import CatWithEval
import warnings
warnings.filterwarnings("ignore")
set_config(enable_metadata_routing=True, skip_parameter_validation=True)
optuna.logging.set_verbosity(optuna.logging.WARNING)

def fit_final_models(main_data: pd.DataFrame,
                    meta_data: pd.DataFrame,
                    full_ds: pd.DataFrame,
                     hp: Dict[str, Any]) -> Tuple[float, float, Any, Any]:
    # Verificar correspondencia entre características y objetivos
    def verify_correspondence(X, y, data, mask, set_name):
        # Obtener índices originales
        original_indices = data[mask].index
        # Seleccionar 5 índices aleatorios
        sample_indices = np.random.choice(len(original_indices), min(5, len(original_indices)), replace=False)
        for idx in sample_indices:
            orig_idx = original_indices[idx]
            # Verificar que los valores coincidan
            if not np.array_equal(X.iloc[idx].values, data.loc[orig_idx, X.columns].values):
                raise ValueError(f"Discrepancia en {set_name} características para índice {idx}")
            if y.iloc[idx] != data.loc[orig_idx, y.name]:
                raise ValueError(f"Discrepancia en {set_name} objetivo para índice {idx}")

    gc.collect()
    # ---------- 1) main model_main ----------
    X_main = main_data.loc[:, main_data.columns.str.contains('_feature') & ~main_data.columns.str.contains('_meta_feature')]
    y_main = main_data['labels'].astype('int16')
    # Check for inf values in main features
    inf_cols_main = X_main.columns[X_main.isin([np.inf, -np.inf]).any()].tolist()
    if inf_cols_main:
        print("Main features with inf values:", inf_cols_main)
    # Check for NaN values in main features
    nan_cols_main = X_main.columns[X_main.isna().any()].tolist()
    if nan_cols_main:
        print("Main features with NaN values:", nan_cols_main)
    # División de datos para el modelo principal según fechas
    main_train_mask = (X_main.index > hp['backward']) & (X_main.index < hp['forward'])
    main_val_mask = (X_main.index > hp['val_backward']) & (X_main.index < hp['val_forward'])
    X_train_main = X_main.loc[main_train_mask].reset_index(drop=True)
    y_train_main = y_main.loc[main_train_mask].reset_index(drop=True)
    X_val_main = X_main.loc[main_val_mask].reset_index(drop=True)
    y_val_main = y_main.loc[main_val_mask].reset_index(drop=True)
    # ── descartar clusters problemáticos ────────────────────────────
    if len(y_train_main.value_counts()) < 2 or len(y_val_main.value_counts()) < 2:
        return None, None, None, None
    # Verificar tanto conjunto de entrenamiento como de validación
    verify_correspondence(X_train_main, y_train_main, main_data, main_train_mask, "entrenamiento MAIN")
    verify_correspondence(X_val_main, y_val_main, main_data, main_val_mask, "validación MAIN")

    # Main model
    cat_main_params = dict(
        # iterations=hp['cat_iterations'],
        # depth=hp['cat_depth'],
        # learning_rate=hp['cat_learning_rate'],
        # l2_leaf_reg=hp['cat_l2_leaf_reg'],
        early_stopping_rounds=hp['cat_early_stopping'],
        eval_metric='Accuracy',
        verbose=False,
        thread_count=-1,
        task_type='CPU',
        used_ram_limit="8gb"
    )
    xgb_main_params = dict(
        # n_estimators=hp['xgb_n_estimators'],
        # max_depth=hp['xgb_max_depth'],
        # learning_rate=hp['xgb_learning_rate'],
        # reg_lambda=hp['xgb_reg_lambda'],
        early_stopping_rounds=hp['xgb_early_stopping'],
        eval_metric='logloss',
        verbosity=0,
        silent=True,
        n_jobs=-1,
        tree_method= "hist",
        device_type="cuda"
    )
    lgbm_main_params = dict(
        # n_estimators=hp['lgbm_n_estimators'],
        # max_depth=hp['lgbm_max_depth'],
        # learning_rate=hp['lgbm_learning_rate'],
        # reg_lambda=hp['lgbm_reg_lambda'],
        early_stopping_round=hp['lgbm_early_stopping'],
        metric='auc',
        verbosity=-1,
        silent=True,
        n_jobs=-1
    )
    base_main_models = [
        ('catboost', CatWithEval(
            **cat_main_params,
            eval_set=[(X_val_main, y_val_main)])),
        ('xgboost', XGBWithEval(
            **xgb_main_params, 
            eval_set=[(X_val_main, y_val_main)])),
        ('lightgbm', LGBMWithEval(
            **lgbm_main_params, 
            eval_set=[(X_val_main, y_val_main)]))
    ]
    model_main = VotingClassifier(
            estimators=base_main_models,
            voting='soft',
            flatten_transform=False,
            n_jobs=1
        )
    model_main.fit(X_train_main, y_train_main)

    gc.collect()
    # ---------- 2) meta‑modelo ----------
    X_meta = meta_data.loc[:, meta_data.columns.str.contains('_meta_feature')]
    y_meta = meta_data['clusters'].astype('int16')
    # Check for inf values in meta features
    inf_cols_meta = X_meta.columns[X_meta.isin([np.inf, -np.inf]).any()].tolist()
    if inf_cols_meta:
        print("Meta features with inf values:", inf_cols_meta)
    # Check for NaN values in meta features
    nan_cols_meta = X_meta.columns[X_meta.isna().any()].tolist()
    if nan_cols_meta:
        print("Meta features with NaN values:", nan_cols_meta)
    # División de datos para el modelo principal según fechas
    meta_train_mask = (X_meta.index > hp['backward']) & (X_meta.index < hp['forward'])
    meta_val_mask = (X_meta.index > hp['val_backward']) & (X_meta.index < hp['val_forward'])
    X_train_meta = X_meta.loc[meta_train_mask].reset_index(drop=True)
    y_train_meta = y_meta.loc[meta_train_mask].reset_index(drop=True)
    X_val_meta = X_meta.loc[meta_val_mask].reset_index(drop=True)
    y_val_meta = y_meta.loc[meta_val_mask].reset_index(drop=True)
    # ── descartar clusters problemáticos ────────────────────────────
    if len(y_train_meta.value_counts()) < 2 or len(y_val_meta.value_counts()) < 2:
        return None, None, None, None
    # Verificar tanto conjunto de entrenamiento como de validación
    verify_correspondence(X_train_meta, y_train_meta, meta_data, meta_train_mask, "entrenamiento META")
    verify_correspondence(X_val_meta, y_val_meta, meta_data, meta_val_mask, "validación META")

    # Meta-modelo
    cat_meta_params = dict(
        # iterations=hp['cat_iterations'],
        # depth=hp['cat_depth'],
        # learning_rate=hp['cat_learning_rate'],
        # l2_leaf_reg=hp['cat_l2_leaf_reg'],
        early_stopping_rounds=hp['cat_early_stopping'],
        eval_metric='F1',
        verbose=False,
        thread_count=-1,
        task_type='CPU',
        used_ram_limit="8gb"
    )
    xgb_meta_params = dict(
        # n_estimators=hp['xgb_n_estimators'],
        # max_depth=hp['xgb_max_depth'],
        # learning_rate=hp['xgb_learning_rate'],
        # reg_lambda=hp['xgb_reg_lambda'],
        early_stopping_rounds=hp['xgb_early_stopping'],
        eval_metric='auc',
        verbosity=0,
        verbose_eval=False,
        silent=True,
        n_jobs=-1,
        tree_method= "hist",
        device_type="cuda"
    )
    lgbm_meta_params = dict(
        # n_estimators=hp['lgbm_n_estimators'],
        # max_depth=hp['lgbm_max_depth'],
        # learning_rate=hp['lgbm_learning_rate'],
        # reg_lambda=hp['lgbm_reg_lambda'],
        early_stopping_round=hp['lgbm_early_stopping'],
        metric='binary_logloss',
        verbosity=-1,
        silent=True,
        n_jobs=-1
    )
    base_meta_models = [
        ('catboost', CatWithEval(
            **cat_meta_params,
            eval_set=[(X_val_meta, y_val_meta)])),
        ('xgboost', XGBWithEval(
            **xgb_meta_params, 
            eval_set=[(X_val_meta, y_val_meta)])),
        ('lightgbm', LGBMWithEval(
            **lgbm_meta_params, 
            eval_set=[(X_val_meta, y_val_meta)]))
    ]

    model_meta = VotingClassifier(
            estimators=base_meta_models,
            voting='soft',
            flatten_transform=False,
            n_jobs=1
        )
    model_meta.fit(X_train_meta, y_train_meta)

    # 4) Evaluación en datos in-sample
    r2_ins = test_model_one_direction(
        full_ds,
        [model_main, model_meta],
        hp['forward'],
        hp['backward'],
        hp['direction'],
        plt=False,
    )

    # 5) Evaluación en datos out-of-sample
    r2_oos = test_model_one_direction(
        full_ds,
        [model_main, model_meta],
        hp['full_forward'],
        hp['forward'],
        hp['direction'],
        plt=False,
    )
    
    # 6) Evaluación en datos históricos (backward)
    r2_val = test_model_one_direction(
        full_ds,
        [model_main, model_meta],
        hp['val_forward'],
        hp['val_backward'],
        hp['direction'],
        plt=False,
    )
    r2_oos = r2_oos * 0.7 + r2_val * 0.3
        
    return r2_ins, r2_oos, model_main, model_meta

def objective(trial: optuna.trial.Trial, base_hp: Dict[str, Any], study=None) -> float:
    # ─── cálculo combinado simple y simétrico ──────────────────────────
    def calc_score(fwd: float, bwd: float, eps: float = 1e-9) -> float:
        if (fwd is None or bwd is None or
            not np.isfinite(fwd) or not np.isfinite(bwd) or
            fwd <= 0 or bwd <= 0):
            return -1.0
        mean = 0.6 * fwd + 0.4 * bwd
        if fwd < bwd * 0.8:
            mean *= 0.8
        delta  = abs(fwd - bwd) / max(abs(fwd), abs(bwd), eps)
        score  = mean * (1.0 - delta)
        return score
    
    gc.collect()
    best_combined_score = -math.inf
    hp = {k: copy.deepcopy(v) for k, v in base_hp.items() if k != 'base_df'}

    # µ··· Espacio de búsqueda optimizado ···µ
    # Parámetros de clustering más amplios para encontrar patrones más diversos
    hp['n_clusters'] = trial.suggest_int('n_clusters', 5, 100, step=5)  
    hp['step'] = trial.suggest_int('step', 1, hp['n_clusters'], log=True)
    hp['label_max'] = trial.suggest_int('label_max', 3, 15)
    hp['markup'] = trial.suggest_float("markup", 0.1, 1.0, step=0.1)
    hp['atr_period'] = trial.suggest_int('atr_period', 10, 100, step=5)
    hp['base_window'] = trial.suggest_int('base_window', hp['n_clusters'], 250, step=10)

    # Main model parameters (same ranges as meta)
    # hp['cat_iterations'] = trial.suggest_int('cat_iterations', 100, 1000, step=100)
    # hp['cat_depth'] = trial.suggest_int('cat_depth', 3, 10)
    # hp['cat_learning_rate'] = trial.suggest_float('cat_learning_rate', 0.01, 0.3, log=True)
    # hp['cat_l2_leaf_reg'] = trial.suggest_float('cat_l2_leaf_reg', 1.0, 10.0, step=0.5)
    hp['cat_early_stopping'] = trial.suggest_int('cat_early_stopping', 50, 500, step=50)
    
    # hp['xgb_n_estimators'] = trial.suggest_int('xgb_n_estimators', 100, 1000, step=100)
    # hp['xgb_max_depth'] = trial.suggest_int('xgb_max_depth', 3, 10)
    # hp['xgb_learning_rate'] = trial.suggest_float('xgb_learning_rate', 0.01, 0.3, log=True)
    # hp['xgb_reg_lambda'] = trial.suggest_float('xgb_reg_lambda', 1.0, 10.0, step=0.5)
    hp['xgb_early_stopping'] = trial.suggest_int('xgb_early_stopping', 50, 500, step=50)
    
    # hp['lgbm_n_estimators'] = trial.suggest_int('lgbm_n_estimators', 100, 1000, step=100)
    # hp['lgbm_max_depth'] = trial.suggest_int('lgbm_max_depth', 3, 10)
    # hp['lgbm_learning_rate'] = trial.suggest_float('lgbm_learning_rate', 0.01, 0.3, log=True)
    # hp['lgbm_reg_lambda'] = trial.suggest_float('lgbm_reg_lambda', 1.0, 10.0, step=0.5)
    hp['lgbm_early_stopping'] = trial.suggest_int('lgbm_early_stopping', 50, 500, step=50)

    # Optimización de períodos para el modelo principal
    n_periods_main = trial.suggest_int('n_periods_main', 5, 15)
    main_periods = []
    for i in range(n_periods_main):
        period_main = trial.suggest_int(f'period_main_{i}', 5, 200, log=True)
        main_periods.append(period_main)
    main_periods = sorted(list(set(main_periods)))
    hp['periods_main'] = main_periods

    # Optimización de períodos para el meta-modelo
    n_periods_meta = 1 #trial.suggest_int('n_periods_meta', 1, 2)
    meta_periods = []
    for i in range(n_periods_meta):
        period_meta = trial.suggest_int(f'period_meta_{i}', 3, 7, log=True)
        meta_periods.append(period_meta)
    meta_periods = sorted(list(set(meta_periods)))
    hp['periods_meta'] = meta_periods

    # Selección de estadísticas para el modelo principal
    main_stat_choices = [
        "std", "skew", "kurt", "zscore", "range", "mad", "entropy", 
        "slope", "momentum", "fractal", "hurst", "autocorr", "max_dd", 
        "sharpe", "fisher", "chande", "var", "approx_entropy", 
        "eff_ratio", "corr_skew", "jump_vol", "vol_skew", "hurst"
    ]

    n_main_stats = trial.suggest_int('n_main_stats', 1, len(main_stat_choices), log=True)
    selected_main_stats = []
    for i in range(n_main_stats):
        stat = trial.suggest_categorical(f'main_stat_{i}', main_stat_choices)
        selected_main_stats.append(stat)
    selected_main_stats = list(set(selected_main_stats))
    if len(selected_main_stats) == 1 and ("fractal" in selected_main_stats or "jump_vol" in selected_main_stats):
        remaining_stats = [s for s in main_stat_choices if s != "fractal" and s != "jump_vol"]
        additional_stat = trial.suggest_categorical('additional_stat', remaining_stats)
        selected_main_stats.append(additional_stat)
    hp["stats_main"] = selected_main_stats
    #print(f"Main features seleccionadas: {hp['stats_main']}")

    # Selección de estadísticas para el meta-modelo
    meta_stat_choices = [
        "std", "skew", "zscore", "range", "mad", "entropy", 
        "slope", "momentum", "autocorr", "max_dd", "sharpe", 
        "eff_ratio", "fisher", "chande", "var", "approx_entropy", 
        "corr_skew", "vol_skew", "hurst", "kurt"
    ]
    # Seleccionar estadísticas meta
    n_meta_stats = trial.suggest_int('n_meta_stats', 1, len(meta_stat_choices), log=True)
    selected_meta_stats = []
    for i in range(n_meta_stats):
        stat = trial.suggest_categorical(f'meta_stat_{i}', meta_stat_choices)
        selected_meta_stats.append(stat)
    selected_meta_stats = list(set(selected_meta_stats))
    hp["stats_meta"] = selected_meta_stats
    #print(f"Meta features seleccionadas: {hp['stats_meta']} | Periodo: {hp['periods_meta']}")

    # Dataset completo
    full_ds = get_features(base_hp['base_df'], hp)
    
    # Verificar valores no válidos
    if full_ds.isna().any().any() or np.isinf(full_ds).any().any():
        print("¡ADVERTENCIA! El dataset contiene valores NaN o infinitos:")
        print("\nValores NaN por columna:")
        print(full_ds.isna().sum())
        print("\nValores infinitos por columna:")
        print(np.isinf(full_ds).sum())
    
    # Determinar la fecha inicial más antigua
    start_date = min(hp['val_backward'], hp['backward'])
    ds_train_mask = (full_ds.index > start_date) & (full_ds.index < hp['forward'])
    ds_train = full_ds[ds_train_mask]

    # Clustering
    ds_train = sliding_window_clustering(
        ds_train,
        n_clusters=hp['n_clusters'],
        step=hp.get('step', None),
        atr_period=hp['atr_period'],
        base_window=hp['base_window']
    )
    # Evaluar clusters ordenados por tamaño
    cluster_sizes = ds_train['clusters'].value_counts()
    # Filtrar el cluster 0 (inválido) si existe
    if 0 in cluster_sizes.index:
        cluster_sizes = cluster_sizes.drop(0)
    for clust in cluster_sizes.index:
        # Main data
        main_data = ds_train.loc[ds_train['clusters'] == clust]
        if len(main_data) <= hp['label_max']:
            continue

        main_data = get_labels_one_direction(
            main_data,
            markup=hp['markup'],
            max_val=hp['label_max'],
            direction=hp['direction'],
            atr_period=hp['atr_period'],
            deterministic=False
        )
        # Meta data
        meta_data = ds_train.copy()
        meta_data['clusters'] = (meta_data['clusters'] == clust).astype(int)

        # Evaluación en ambos períodos
        r2_ins, r2_oos, model_main, meta_model = fit_final_models(
            main_data,
            meta_data,
            full_ds,
            hp
        )
        if r2_ins == None or r2_oos == None or model_main == None or meta_model == None:
            continue
        # Calcular puntuación combinada (puedes ajustar los pesos según necesites)
        score = calc_score(r2_ins, r2_oos)
        if score <= -1.0:
            continue  

        if score > best_combined_score:
            best_combined_score = score
            # Guardar información del trial actual
            trial.set_user_attr("r2_ins", r2_ins)
            trial.set_user_attr("r2_oos", r2_oos)
            trial.set_user_attr("combined_score", score)
            trial.set_user_attr("cluster_id", clust)
            trial.set_user_attr("stats_main", hp["stats_main"])
            trial.set_user_attr("stats_meta", hp["stats_meta"])
            trial.set_user_attr("periods_main", hp["periods_main"])
            trial.set_user_attr("periods_meta", hp["periods_meta"])
            # Guardar parámetros del trial actual (sin fechas)
            params_to_save = hp.copy()
            params_to_save.pop('backward', None)
            params_to_save.pop('forward', None)
            params_to_save.pop('full_forward', None)
            trial.set_user_attr("params", params_to_save)
            # Si existe el estudio, actualizar sus atributos
            if study is not None:
                current_best = study.user_attrs.get("best_combined_score", -math.inf)
                if score > current_best:
                    study.set_user_attr("best_params", params_to_save)
                    study.set_user_attr("best_metrics", {
                        "r2_ins": r2_ins,
                        "r2_oos": r2_oos,
                        "combined_score": score,
                        "cluster_id": clust,
                    })
                    study.set_user_attr("best_combined_score", score)
                    study.set_user_attr("best_models", [model_main, meta_model])
                    study.set_user_attr("best_stats_main", hp["stats_main"])
                    study.set_user_attr("best_stats_meta", hp["stats_meta"])
                    study.set_user_attr("best_periods_main", hp["periods_main"])
                    study.set_user_attr("best_periods_meta", hp["periods_meta"])
                    study.set_user_attr("best_trial_number", trial.number)
                    study.set_user_attr("best_df_sample", full_ds.sample(1))
    # Si no hay ningún cluster válido, devolver un valor negativo pero no infinito
    if best_combined_score == -math.inf:
        return -10.0

    # No aplicar penalización adicional por pocos clusters para mantener coherencia
    return best_combined_score

def optimize_and_export(base_hp: Dict[str, Any]):
    """Lanza Optuna, guarda el mejor modelo y lo exporta a ONNX."""
    def show_best_summary(study: optuna.study.Study, model_seed) -> None:
        """Muestra en un único cuadro el resultado óptimo del estudio."""
        # ── extraer métricas y modelos ───────────────────────────────────────
        m  = study.user_attrs.get("best_metrics", {})
        f2 = m.get("r2_ins",   float("nan"))
        b2 = m.get("r2_oos",  float("nan"))
        c2 = m.get("combined_score", float("nan"))
        best_trial = study.user_attrs.get("best_trial_number", "-")
        best_df_sample = study.user_attrs.get("best_df_sample", None)

        main_cls = meta_cls = "None"
        best_models = study.user_attrs.get("best_models")
        if best_models and len(best_models) == 2:
            main_cls = type(best_models[0]).__name__
            meta_cls = type(best_models[1]).__name__

        # ── cuadro de resumen ────────────────────────────────────────────────
        lines = [
            "┌" + "─" * 55 + "┐",
            f"│  MODELO {model_seed} TRIAL ÓPTIMO #{best_trial}",
            "├" + "─" * 55 + "┤",
            f"│  R² INS : {f2:10.4f}  │  R² OOS : {b2:10.4f} │",
            f"│  Combined     : {c2:10.4f}                            │",
            "├" + "─" * 55 + "┤"
        ]
        print("\n".join(lines))

    base_hp.update({
        'model_seed': random.randint(0, 10000000),
    })
    # Configurar el pruner inteligente
    # pruner = SuccessiveHalvingPruner(
    #     min_resource=1,
    #     reduction_factor=3,
    #     min_early_stopping_rate=0
    # )

    # Crear el estudio sin persistencia
    study = optuna.create_study(
        direction='maximize',
        pruner=HyperbandPruner(),
        sampler=optuna.samplers.TPESampler(
            n_startup_trials=int(np.sqrt(base_hp['n_trials'])),
        )
    )
    study.optimize(lambda t: objective(t, base_hp, study),
                n_trials=base_hp['n_trials'],
                show_progress_bar=True)

    # ── verificación y cuadro ───────────────────────────────
    best_trial = study.best_trial
    assert study.user_attrs.get("best_trial_number") == best_trial.number, \
        "best_trial_number no coincide con study.best_trial.number"

    best_metric_saved = study.user_attrs.get("best_metrics", {}).get("combined_score")
    if best_metric_saved is not None:
        assert math.isclose(best_metric_saved, study.best_value, rel_tol=1e-9), \
            "combined_score guardado ≠ study.best_value"
    else:
        print("⚠️  Ningún trial produjo un score válido; modelos no guardados.")

    best_models = study.user_attrs.get("best_models")
    assert best_models and len(best_models) == 2 and all(best_models), \
        "best_models incorrecto"

    show_best_summary(study, base_hp['model_seed'])

    # ── exportación ─────────────────────────────────────────
    export_params = base_hp.copy()
    export_params.update({
        "best_trial": study.user_attrs["best_trial_number"],
        "best_score": study.user_attrs["best_combined_score"],
        "best_periods_main": study.user_attrs["best_periods_main"],
        "best_periods_meta": study.user_attrs["best_periods_meta"],
        "best_stats_main"  : study.user_attrs["best_stats_main"],
        "best_stats_meta"  : study.user_attrs["best_stats_meta"],
        "best_models"      : study.user_attrs["best_models"],
    })
    export_model_to_ONNX(**export_params)
    # ── devolver métricas ─────────────────────────────────────────
    return {
        "r2_ins"   : study.user_attrs.get("best_metrics", {}).get("r2_ins",   float("nan")),
        "r2_oos"  : study.user_attrs.get("best_metrics", {}).get("r2_oos",  float("nan")),
        "combined_score": study.user_attrs.get("best_metrics", {}).get("combined_score", float("nan")),
    }

if __name__ == "__main__":
    base_hp: Dict[str, Any] = {
        'symbol': r'XAUUSD',
        'timeframe': 'H1',
        'direction': 'buy',
        'val_backward': datetime(2018, 1, 1),
        'val_forward': datetime(2019, 1, 1),
        'backward': datetime(2019, 1, 1),
        'forward': datetime(2024, 1, 1),
        'full_forward': datetime(2025, 1, 1),
        'model_seed': 0,
        'n_trials': 1000,
        'models_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files/',
        'include_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Include/ajmtrz/include/Dmitrievsky',
        'history_path': r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/",
        'stats_main': [],
        'stats_meta': [],
        'best_models': [None, None],
        'markup': 0.20,
        'label_min'  : 1,
        'label_max'  : 15,
        'n_clusters': 30,
        'window_size': 350,
        'periods_main': [i for i in range(5, 300, 30)],
        'periods_meta': [5],
    }
    base_df = get_prices(base_hp)
    base_hp.update({
        'base_df': base_df,
    })
    # Para recopilar resultados globales de todos los modelos
    all_results = {}
    best_models = []
    model_range = range(0, 5)
    base_hp.update({
        'n_trials': 500,
    })
    for i in tqdm(model_range, desc=f"Optimizando {base_hp['symbol']}/{base_hp['timeframe']}", unit="modelo"):
        try:
            model_results = optimize_and_export(base_hp)
            best_models.append((i, model_results))
            
            # Añadir a resultados globales
            all_results[f"model_{i}"] = {
                "success": True,
                "r2_ins": model_results["r2_ins"],
                "r2_oos": model_results["r2_oos"],
                "combined_score": model_results["combined_score"]
            }
            
        except Exception as e:
            import traceback
            tqdm.write(f"\nError procesando modelo {i}: {str(e)}")
            tqdm.write(traceback.format_exc())
            
            all_results[f"model_{i}"] = {
                "success": False,
                "error": str(e)
            }
            continue
    
    # Resumen final
    print("\n" + "="*50)
    print(f"RESUMEN DE OPTIMIZACIÓN {base_hp['symbol']}/{base_hp['timeframe']}")
    print("="*50)
    
    successful_models = [info for model_key, info  in all_results.items() if info.get("success", False)]
    print(f"Modelos completados exitosamente: {len(successful_models)}/{len(model_range)}")
    
    if successful_models:
        # Calcular estadísticas globales
        r2_ins_scores = [info["r2_ins"] for info in successful_models]
        r2_oos_scores = [info["r2_oos"] for info in successful_models]
        combined_scores = [info["combined_score"] for info in successful_models]
        
        print(f"\nEstadísticas de rendimiento:")
        print(f"  R2 INS promedio: {np.mean(r2_ins_scores):.4f} ± {np.std(r2_ins_scores):.4f}")
        print(f"  R2 OOS promedio: {np.mean(r2_oos_scores):.4f} ± {np.std(r2_oos_scores):.4f}")
        print(f"  Puntuación combinada promedio: {np.mean(combined_scores):.4f} ± {np.std(combined_scores):.4f}")

        # Identificar el mejor modelo global basado en la puntuación combinada
        successful = [(k, v) for k, v in all_results.items() if v.get("success", False)]
        combined_scores = [v["combined_score"] for _, v in successful]
        best_model_key, best_info = successful[int(np.argmax(combined_scores))]
        
        print(f"\nMejor modelo global: {best_model_key}")
        print(f"  R2 INS: {best_info['r2_ins']:.4f}")
        print(f"  R2 OOS: {best_info['r2_oos']:.4f}")
        print(f"  Puntuación combinada: {best_info['combined_score']:.4f}")
    
    print("\nProceso de optimización completado.")

Optimizando XAUUSD/H1:   0%|          | 0/5 [00:00<?, ?modelo/s]

  0%|          | 0/500 [00:00<?, ?it/s]